# Editing Software

In [76]:
import datetime
import json

import duckdb
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import util
from IPython.display import HTML

pio.renderers.default = "plotly_mimetype"
duckdb.execute("SET enable_progress_bar = false")
pd.set_option("display.max_rows", 100)

default_layout = dict(
    margin=dict(l=55, r=55, b=55, t=55),
    font=dict(family="Times", size=15),
    title_x=0.5,
    paper_bgcolor="#f5f2f0",
    plot_bgcolor="#f5f2f0",
    xaxis=dict(tickcolor="black", linecolor="black", showgrid=True, gridcolor="darkgray", zerolinecolor="darkgray"),
    yaxis=dict(
        tickcolor="black",
        linecolor="black",
        showgrid=True,
        gridcolor="darkgray",
        zerolinecolor="darkgray",
        rangemode="tozero",
    ),
)

display(
    HTML(
        '<link rel="stylesheet" type="text/css" href="../notebooks/notebook.css"><script src="../notebooks/notebook.js"></script>'
    )
)

## Top 10 Editing Software

In [98]:
def add_traces_to_figure_return_button(fig, df, title, y_col, y_axis_title, software_list, visible, total_traces):
    """Add traces to figure and return button configuration."""
    current_trace_count = len(fig.data)

    for software in software_list:
        software_data = df[df["created_by"] == software]
        fig.add_trace(
            go.Scatter(
                x=software_data["months"],
                y=software_data[y_col],
                name=software,
                visible=visible,
                hovertemplate=f"{software}" + f"<br>%{{x}}<br>%{{y:,}} {y_col}<extra></extra>",
            )
        )

    # Create visibility array for this button
    visibility = [False] * total_traces
    for i in range(current_trace_count, current_trace_count + len(software_list)):
        visibility[i] = True

    return {
        "label": y_col.replace("_", " ").title(),
        "args": [{"visible": visibility}, {"title.text": title, "yaxis.title.text": y_axis_title}],
        "method": "update",
    }


def get_top_software(limit=10):
    """Get top editing software by total contributors."""
    return (
        duckdb.sql(f"""
        WITH software_total_contributors AS (
            SELECT
                created_by,
                COUNT(DISTINCT user_name) as total_contributors
            FROM '../data_enriched/year=*/month=*/*.parquet'
            WHERE created_by IS NOT NULL
            GROUP BY created_by
            ORDER BY total_contributors DESC
            LIMIT {limit}
        )
        SELECT created_by FROM software_total_contributors
        """)
        .df()["created_by"]
        .tolist()
    )


def add_monthly_contributors_to_figure(fig, software_list, visible, total_traces):
    """Get monthly contributors for specified software."""
    software_filter = ",".join([f"'{s}'" for s in software_list])
    df = duckdb.sql(f"""
    SELECT 
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        created_by,
        COUNT(DISTINCT user_name) as "Contributors"
    FROM '../data_enriched/year=*/month=*/*.parquet'
    WHERE created_by IN ({software_filter})
    GROUP BY year, month, created_by
    ORDER BY year, month, created_by
    """).df()[: -len(software_list)]  # Remove last month for each software (incomplete data)
    title = f"Monthly Contributors by Top {len(software_list)} Editing Software"
    return add_traces_to_figure_return_button(
        fig, df, title, "Contributors", "Contributors", software_list, visible, total_traces
    )


def add_monthly_new_contributors_to_figure(fig, software_list, visible, total_traces):
    """Get monthly new contributors for specified software."""
    software_filter = ",".join([f"'{s}'" for s in software_list])
    df = duckdb.sql(f"""
    WITH user_first_appearance AS (
        SELECT 
            user_name,
            year,
            month,
            created_by,
            ROW_NUMBER() OVER (PARTITION BY user_name ORDER BY year, month) as rn
        FROM (
            SELECT DISTINCT user_name, year, month, created_by
            FROM '../data_enriched/year=*/month=*/*.parquet'
            WHERE created_by IN ({software_filter})
        )
    ),
    first_appearances AS (
        SELECT user_name, year, month, created_by
        FROM user_first_appearance 
        WHERE rn = 1
    )
    SELECT
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        created_by,
        COUNT(DISTINCT user_name) as "New Contributors"
    FROM first_appearances
    GROUP BY year, month, created_by
    ORDER BY year, month, created_by
    """).df()[: -len(software_list)]
    title = f"Monthly New Contributors by Top {len(software_list)} Editing Software"
    return add_traces_to_figure_return_button(
        fig, df, title, "New Contributors", "New Contributors", software_list, visible, total_traces
    )


def add_monthly_edit_count_to_figure(fig, software_list, visible, total_traces):
    """Get monthly edit count for specified software."""
    software_filter = ",".join([f"'{s}'" for s in software_list])
    df = duckdb.sql(f"""
    SELECT 
        CONCAT(year, '-', LPAD(CAST(month as VARCHAR), 2, '0')) as months,
        created_by,
        SUM(edit_count) as "Edit Count"
    FROM '../data_enriched/year=*/month=*/*.parquet'
    WHERE created_by IN ({software_filter})
    GROUP BY year, month, created_by
    ORDER BY year, month, created_by
    """).df()[: -len(software_list)]  # Remove last month for each software (incomplete data)
    title = f"Monthly Edit Count by Top {len(software_list)} Editing Software"
    return add_traces_to_figure_return_button(
        fig, df, title, "Edit Count", "Edit Count", software_list, visible, total_traces
    )


top_n = 10
top_software = get_top_software(top_n)
fig = go.Figure()
buttons = []

buttons.append(
    add_monthly_contributors_to_figure(
        fig,
        software_list=top_software,
        visible=True,
        total_traces=3 * top_n,
    )
)
buttons.append(
    add_monthly_new_contributors_to_figure(
        fig,
        software_list=top_software,
        visible=False,
        total_traces=3 * top_n,
    )
)
buttons.append(
    add_monthly_edit_count_to_figure(
        fig,
        software_list=top_software,
        visible=False,
        total_traces=3 * top_n,
    )
)

fig.update_layout(
    title=buttons[0]["args"][1]["title.text"],
    xaxis_title="Month",
    yaxis_title=buttons[0]["args"][1]["yaxis.title.text"],
    updatemenus=[{"type": "buttons", "buttons": buttons}],
    **default_layout,
)
fig.show()

## Top 100 Editing Software Yearly

In [90]:
def get_last_years_sql_filter(last_years):
    current_year = datetime.datetime.now().year
    years_to_include = list(range(current_year - last_years + 1, current_year + 1))
    year_list = ", ".join(str(year) for year in years_to_include)
    return f"AND year IN ({year_list})"


def get_contributors_count(name_to_html_link, year_filter=""):
    query = f"""
    WITH top_100_software AS (
        SELECT
            created_by as "Editing Software",
            CAST(COUNT(DISTINCT user_name) as BIGINT) as total_contributors
        FROM '../data_enriched/year=*/month=*/*.parquet'
        WHERE created_by IS NOT NULL
            {year_filter}
        GROUP BY created_by
        ORDER BY total_contributors DESC
        LIMIT 100
    ),
    software_yearly_contributors AS (
        SELECT
            year,
            created_by as "Editing Software",
            CAST(COUNT(DISTINCT user_name) as BIGINT) as contributors
        FROM '../data_enriched/year=*/month=*/*.parquet'
        WHERE created_by IS NOT NULL
            AND created_by IN (SELECT "Editing Software" FROM top_100_software)
            {year_filter}
        GROUP BY year, created_by
    ),
    yearly_data AS (
        SELECT
            syc.year,
            syc."Editing Software",
            syc.contributors,
            t100.total_contributors
        FROM software_yearly_contributors syc
        JOIN top_100_software t100
            ON syc."Editing Software" = t100."Editing Software"
    ),
    pivoted_data AS (
        SELECT 
            "Editing Software",
            total_contributors as "Total",
            * EXCLUDE ("Editing Software", total_contributors)
        FROM (
            PIVOT yearly_data
            ON year
            USING SUM(contributors)
        )
    )
    SELECT 
        ROW_NUMBER() OVER (ORDER BY "Total" DESC) as "Rank",
        "Editing Software",
        * EXCLUDE ("Editing Software", "Total"),
        "Total"
    FROM pivoted_data
    ORDER BY "Total" DESC
    """
    df = duckdb.sql(query).df().fillna(0)
    df["Editing Software"] = df["Editing Software"].apply(
        lambda name: name_to_html_link[name] if name in name_to_html_link else name
    )
    return df


def get_new_contributors(name_to_html_link, year_filter=""):
    query = f"""
    WITH user_first_year AS (
        SELECT 
            user_name,
            created_by,
            MIN(year) as first_year
        FROM '../data_enriched/year=*/month=*/*.parquet'
        WHERE created_by IS NOT NULL
        GROUP BY user_name, created_by
    ),
    top_100_software AS (
        SELECT
            created_by as "Editing Software",
            CAST(COUNT(DISTINCT user_name) as BIGINT) as total_new_contributors
        FROM user_first_year
        WHERE created_by IS NOT NULL
            {year_filter.replace("year", "first_year") if year_filter else ""}
        GROUP BY created_by
        ORDER BY total_new_contributors DESC
        LIMIT 100
    ),
    software_yearly_new_contributors AS (
        SELECT
            first_year as year,
            created_by as "Editing Software",
            CAST(COUNT(DISTINCT user_name) as BIGINT) as new_contributors
        FROM user_first_year
        WHERE created_by IS NOT NULL
            AND created_by IN (SELECT "Editing Software" FROM top_100_software)
            {year_filter.replace("year", "first_year") if year_filter else ""}
        GROUP BY first_year, created_by
    ),
    yearly_data AS (
        SELECT
            sync.year,
            sync."Editing Software",
            sync.new_contributors,
            t100.total_new_contributors
        FROM software_yearly_new_contributors sync
        JOIN top_100_software t100
            ON sync."Editing Software" = t100."Editing Software"
    ),
    pivoted_data AS (
        SELECT 
            "Editing Software",
            total_new_contributors as "Total",
            * EXCLUDE ("Editing Software", total_new_contributors)
        FROM (
            PIVOT yearly_data
            ON year
            USING SUM(new_contributors)
        )
    )
    SELECT 
        ROW_NUMBER() OVER (ORDER BY "Total" DESC) as "Rank",
        "Editing Software",
        * EXCLUDE ("Editing Software", "Total"),
        "Total"
    FROM pivoted_data
    ORDER BY "Total" DESC
    """
    df = duckdb.sql(query).df().fillna(0)
    df["Editing Software"] = df["Editing Software"].apply(
        lambda name: name_to_html_link[name] if name in name_to_html_link else name
    )
    return df


def get_edit_count(name_to_html_link, year_filter=""):
    query = f"""
    WITH top_100_software AS (
        SELECT
            created_by as "Editing Software",
            CAST(SUM(edit_count) as BIGINT) as total_edits
        FROM '../data_enriched/year=*/month=*/*.parquet'
        WHERE created_by IS NOT NULL
            {year_filter}
        GROUP BY created_by
        ORDER BY total_edits DESC
        LIMIT 100
    ),
    software_yearly_edits AS (
        SELECT
            year,
            created_by as "Editing Software",
            CAST(SUM(edit_count) as BIGINT) as edits
        FROM '../data_enriched/year=*/month=*/*.parquet'
        WHERE created_by IS NOT NULL
            AND created_by IN (SELECT "Editing Software" FROM top_100_software)
            {year_filter}
        GROUP BY year, created_by
    ),
    yearly_data AS (
        SELECT
            sye.year,
            sye."Editing Software",
            sye.edits,
            t100.total_edits
        FROM software_yearly_edits sye
        JOIN top_100_software t100
            ON sye."Editing Software" = t100."Editing Software"
    ),
    pivoted_data AS (
        SELECT 
            "Editing Software",
            total_edits as "Total",
            * EXCLUDE ("Editing Software", total_edits)
        FROM (
            PIVOT yearly_data
            ON year
            USING SUM(edits)
        )
    )
    SELECT 
        ROW_NUMBER() OVER (ORDER BY "Total" DESC) as "Rank",
        "Editing Software",
        * EXCLUDE ("Editing Software", "Total"),
        "Total"
    FROM pivoted_data
    ORDER BY "Total" DESC
    """
    df = duckdb.sql(query).df().fillna(0)
    df["Editing Software"] = df["Editing Software"].apply(
        lambda name: name_to_html_link[name] if name in name_to_html_link else name
    )
    return df


with open("../config/replace_rules_created_by.json") as f:
    name_to_html_link = {
        name: f'<a href="{item["link"]}">{name}</a>' for name, item in json.load(f).items() if "link" in item
    }

tables = {
    "Contributors Count": get_contributors_count(name_to_html_link, get_last_years_sql_filter(5)),
    "Contributors Count All Years": get_contributors_count(name_to_html_link),
    "New Contributors": get_new_contributors(name_to_html_link, get_last_years_sql_filter(5)),
    "New Contributors All Years": get_new_contributors(name_to_html_link),
    "Edit Count": get_edit_count(name_to_html_link, get_last_years_sql_filter(5)),
    "Edit Count All Years": get_edit_count(name_to_html_link),
}

display(HTML(util.get_tables_html(tables, center_columns=["Rank", "Editing Software"])))

Rank,Editing Software,2021,2022,2023,2024,2025,Total
1,iD,"239,537","204,579","210,136","205,427","108,469","727,487"
2,StreetComplete,"21,935","23,525","26,809","27,111","17,443","65,917"
3,MAPS.ME,"19,974","15,515","16,231","7,729","1,971","51,092"
4,JOSM,"21,512","20,537","19,741","18,127","11,346","46,962"
5,Organic Maps,"3,017","6,258","10,557","12,765","9,308","31,215"
6,OsmAnd,"7,959","8,623","8,544","7,875","4,082","21,869"
7,Rapid,"2,124","2,183","9,121","3,662","2,801","16,283"
8,Vespucci,"4,539","5,025","5,371","5,322","3,080","13,532"
9,Go Map!!,"3,748","3,720","3,830","4,162","2,568","10,197"
10,Map builder,1,"1,506","1,722","6,333",0,"9,399"
